### Lux is now supporting postgreSQL database (April 2021)

In the tutorial, we describe how Lux can be used with data stored in a Postgres database. Postgres is a popular relational databases used by data professionals. By the end of the tutorial, you should be able to: 

- Query data using lux on PostgreSQL backend directly
- Gain a quick overview of the data inside your Postgres database
- Analyze relationships between attributes
- Create custom visualizations
- Export your visualizations to share with others

**Note that SQL support is still experimental and undergoing development. If you are excited about the SQL feature and there is something that you would like to see, please let us know by submitting a [Github Issue here](https://github.com/lux-org/lux/issues/).**

In [ ]:
import lux
import pandas as pd
# Collecting basic usage statistics for Lux (For more information, see: https://tinyurl.com/logging-consent)
lux.logger = True # Remove this line if you do not want your interactions recorded

## Postgres Setup for Lux
0. Setting up Postgres Locally (Optional)
1. Connect to the database
2. Create a `LuxSQLTable` to connect to database table

## 0)  Setting up Postgres Locally (Optional)
If you are accessing this notebook through a live notebook instance on Binder, we have already set up a PostgreSQL database and populated the ```employee``` table for the purpose of this demo, so you can skip this section. 
If you are running this notebook locally and don't have PostgreSQL server setup, please follow these instructions to setup the database for this tutorial locally.

First, install Postgres [here](https://www.postgresql.org) on your system. Next, you can run [this code](https://github.com/lux-org/lux-binder-sql/blob/master/init_db/postgres/init_db.sh) to set up the Postgres database for the demo. Finally, to upload the demo dataset to your database, you can run [this script](https://github.com/lux-org/lux-binder-sql/blob/master/scripts/upload_data.py).


## 1) Create PostgreSQL and connect to the database table

After setting up the Postgres database, we need to create a database connection to our `testdb` database.

In [ ]:
import psycopg2
connection = psycopg2.connect("host=localhost dbname=testdb user=testuser password=testpass")

Alternatively, you can also specify the connection via [sqlalchemy](https://www.sqlalchemy.org/).

Once this connection is created, users can connect Lux to the database using the `set_SQL_connection` command.

In [ ]:
lux.config.set_SQL_connection(connection)

## 2) Create LuxSQLTable object

We create a `LuxSQLTable` as a variable named ```tbl``` and specify the table we are using in the demo to be ```employee```.  The `employee` table contains the [IBM HR Employee Attrition dataset](https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset), including employee characteristics, 
such as number of years spent working at the company, age, and salary rates, might influence attrition. `LuxSQLTable`s can be connected to individual tables or views created within your Postgres database. This can be done by specifying the table or view name in the constructor. 

In [ ]:
tbl = lux.LuxSQLTable()
tbl.set_SQL_table("employee")

The setup is now complete! Now let's explore the `employee` database table using Lux!

<div>
    <img align="left" src="https://github.com/lux-org/lux-resources/blob/master/icon/table.png?raw=True" width="30">
    <h1 style="padding-left: 40px;">Visualizations of database tables</h1>
</div>

To get an overview of the database table, simply print out the LuxSQLTable `tbl`. By clicking on the Toggle button, you can now explore the data visually through Lux. You should see three tabs of visualizations recommended to you. 

In [ ]:
tbl

The visualizations are displayed in different tabs as [actions](https://lux-api.readthedocs.io/en/latest/source/getting_started/overview.html#visualizing-dataframes-with-recommendations).
By inspecting the Correlation action, we see several salient patterns with a sharp triangular pattern. This checks out with our intuition that it is impossible for total working years to exceed the employee's age, and for any employee to stay longer at a company than their total working years.  

<div>
    <img align="left" src="https://github.com/lux-org/lux-resources/blob/master/icon/steering-wheel.png?raw=True" width="30">
    <h1 style="padding-left: 40px;">Steering analysis with intent</h1>
</div>

Let's say that we want to investigate factors that influence employee attrition. Beyond these basic recommendations, you can further specify your analysis *intent*, i.e., the data attributes and values that you are interested in visualizing. 

In [ ]:
tbl.intent=["Attrition"]

Upon printing the dataframe again, Lux leverages the analysis intent to steer the recommendations towards what the user might be interested in.

In [ ]:
tbl

On the left, we see that the visualization based on the specified intent shows that around 15% of employees leaves the company. On the right, in the Enhance action, we learn that employees that leave typically spent less time in their working life and at the Company than their counterparts by about four years. 

ℹ️ Check out [this tutorial](https://lux-api.readthedocs.io/en/latest/source/guide/intent.html#) to learn more about how to specify intent in Lux.

ℹ️ Check out [this tutorial](https://lux-api.readthedocs.io/en/latest/source/guide/vis.html) to learn more about how to create Vis and VisList in Lux.

<div>
    <img align="left" style="margin-top: -10px;" src="https://github.com/lux-org/lux-resources/blob/master/icon/present.png?raw=True" width="50">
    <h1 style="padding-left: 55px;">Exporting visualization insight to edit and share</h1>
</div>

By skimming through the recommended visualizations again, we find that distance from home to work is a important factor in employee attrition.

In [ ]:
tbl

We can scroll down the recommended visualizations and noticed that employees that live further away have higher rates of attrition. We click on the bar chart visualization of `DistanceFromHome` v.s. `Attrition` and the export button on the top right corner, to export this visualization as code.

<img src="https://raw.githubusercontent.com/lux-org/lux-resources/master/doc_img/employee-export.png"></img>

In [ ]:
vis = tbl.exported[0]
print(vis.to_code(language="altair"))

Then we can make minor stylistic changes before we can export this visualization to a slidedeck to share with colleagues.

In [ ]:
import altair as alt
visData = pd.DataFrame({'Attrition': {0: 'No', 1: 'Yes'}, 
                        'DistanceFromHome': {0: 8.915652879156529, 1: 10.632911392405063}})

chart = alt.Chart(visData,title="Important Factor for Employee Attrition!!").mark_bar().encode(
    y = alt.Y('Attrition', type= 'nominal', axis=alt.Axis(labelOverlap=True, title='Attrition')),
    x = alt.X('DistanceFromHome', type= 'quantitative', axis=alt.Axis(title='Average Distance from Home')),
)
chart = chart.configure_mark(tooltip=alt.TooltipContent('encoding'))
chart = chart.configure_title(fontWeight=500,fontSize=13,font='Helvetica Neue')
chart = chart.configure_axis(titleFontWeight=500,titleFontSize=11,titleFont='Helvetica Neue',
            labelFontWeight=400,labelFontSize=8,labelFont='Helvetica Neue',labelColor='#505050')
chart = chart.configure_legend(titleFontWeight=500,titleFontSize=10,titleFont='Helvetica Neue',
            labelFontWeight=400,labelFontSize=8,labelFont='Helvetica Neue')
chart = chart.properties(width=250,height=70)

chart

ℹ️ Check out [this tutorial](https://lux-api.readthedocs.io/en/latest/source/guide/export.html) to learn more about exporting visualizations in Lux.


# Try out Lux! 

To get started, Lux can be installed through [PyPI](https://pypi.org/project/lux-api/). 

```bash
pip install lux-api
``` 


To use Lux in [Jupyter notebook](https://github.com/jupyter/notebook) or [VSCode](https://code.visualstudio.com/docs/python/jupyter-support), activate the notebook extension:

```bash
jupyter nbextension install --py luxwidget
jupyter nbextension enable --py luxwidget
```

To use Lux in [Jupyter Lab](https://github.com/jupyterlab/jupyterlab), activate the lab extension:

```bash
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install luxwidget
```

If you encounter issues with the installation, please refer to [this page](https://lux-api.readthedocs.io/en/latest/source/guide/FAQ.html#troubleshooting-tips) to troubleshoot the installation.

### More information: 

- Follow us on [Twitter](https://twitter.com/lux_api) for discussion and updates.
- Sign up for the early-user [mailing list](https://forms.gle/XKv3ejrshkCi3FJE6) to stay tuned for upcoming releases, updates, or user studies. 
- Visit [ReadTheDocs](https://lux-api.readthedocs.io/en/latest/) for more detailed documentation.
- Report any bugs, issues, or requests through [Github Issues](https://github.com/lux-org/lux/issues). 

<div style="color:#bfbfbf">Icons made by <a href="https://www.freepik.com" title="Freepik" style="color:#b1bbe6">Freepik</a> from <a href="https://www.flaticon.com/" title="Flaticon" style="color:#b1bbe6">www.flaticon.com</a></div>